# In-Class Assignment: Homography


<img src="https://lh5.googleusercontent.com/T7SNdiXctkIi2iWHjjdAZCRBASzRjysFkmivxtD8cZ80QlKFPjOeYa--uiDoHEUdb7daadZQ-fW-l0SVBQJDj90vPFaCKke9JSf6jf8Rk71zpy62TP_hmBpdFONBoA=w740" width="100%">

### Agenda for today's class:


1. Another Homography Example
1. Useful Tools for Image and Video Manipulation
1. Video Registration

----
# 1.  Homography Example

The following code downloads three images. We would like to pick points and evaluate the homography transform similar to the pre-class assignment.  

This example uses some experimental code designed to enable the selecting of points inside an image.  To get this code to work you will need to install the ```mpld3``` library and add the installation path to the notebook. 

**&#9989; DO THIS:** Remove the comment (#) from the following line of code and run the cell to install mpld3.  You only need to do this once so make sure you put the comment back in:


In [ ]:
# The following command will install mpld3 - Uncomment and run once.
!pip install -t ./packages mpld3 

**&#9989; DO THIS:** Next, run the following cells to add the folder where we installed ```mpld3``` download the images and start the point picking program.

In [ ]:
#Add Packages folder to path (Do only once per run of notebook:
from sys import path
package_folder = './packages'
if not package_folder in path:
    print('adding packages')
    path.append(package_folder)
else:
    print('found packages folder')

In [ ]:
#The following code snip-it downloads a file from internet and saves it to your local directory.
from urllib.request import urlopen, urlretrieve
import scipy.misc as misc

urls = []
files = []
ims = []

urls.append('https://goo.gl/93bGGW')
files.append('./Images/Digit1.png')

urls.append('https://goo.gl/GbPB9S')
files.append('./Images/Digit2.png')

urls.append('https://goo.gl/ErQt14')
files.append('./Images/Digit3.png')

for url,file in zip(urls,files):
    urlretrieve(url, file);
    ims.append(misc.imread(file))

In [ ]:
#pick a point
im = ims[0]

In [ ]:
# Code snipit to make point seleciton function (REQUIRES mpld3 installed)

#NOTE this is highly experimental code and may not work as expected on your computer.

%matplotlib inline 
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import sys

import mpld3
from mpld3 import plugins


# MAIN CLASS THAT CONTAINS JAVASCRIPT CODE TO CREATE CIRCLES AND DRAG CIRCLES  
class Annotate(plugins.PluginBase):
    """A plugin that creates points in a figure by clicking the mouse"""
   
    JAVASCRIPT = r"""
    mpld3.register_plugin("annotate", Annotate);
    Annotate.prototype = Object.create(mpld3.Plugin.prototype);
    Annotate.prototype.constructor = Annotate;
    Annotate.prototype.requiredProps = [];
    Annotate.prototype.defaultProps = {radius: 4, color: "white", x: 'x', y: 'y'};
    function Annotate(fig, props){
        mpld3.Plugin.call(this, fig, props);
    };

    Annotate.prototype.draw = function(){
        
        /// NECESSARY STARTUP VARIABLES ///
        
        var fig = this.fig;
        var ax = fig.axes;
        var dataset = [];
        var svg = d3.select(".mpld3-figure");   // existing svg element
        var radius = this.props.radius;
        var color = this.props.color;
        var x = this.props.x;
        var y = this.props.y;
        var ax = fig.axes[0];
        
        
        /// INDEXES HTML DOC TO PULL VALUES FOR x,y CALIBRATION ///
        var xcal = this.parent.axes[0].position[0];
        var ycal = this.parent.axes[0].position[1];
        console.log('x calibration: ' + xcal);
        console.log('y calibration: ' + ycal);
        
        var xcommand = x+" = []";
        IPython.notebook.kernel.execute(xcommand);
        var ycommand = y+" = []";
        IPython.notebook.kernel.execute(ycommand);
        
        
        ////////// CREATE POINT COMPONENT //////////
        
        var update_coords = function() {
        
            return function() {
                var pos = d3.mouse(this),
                    xpos = ax.x.invert(pos[0]),
                    ypos = ax.y.invert(pos[1]);
                    
                var newpoint = {
                    cx: pos[0] + xcal,
                    cy: pos[1] + ycal,
                    r: radius,
                    fill: color
                };
                dataset.push(newpoint);
                
                var circles = svg.selectAll("circle")
                    .data(dataset)
                    .enter()
                    .append("circle")
                    .attr(newpoint)
                    .call(drag);
                       
                var xcommand = x+".append("+xpos+")";
                IPython.notebook.kernel.execute(xcommand);
                console.log(xcommand);
                var ycommand = y+".append("+ypos+")";
                IPython.notebook.kernel.execute(ycommand);
                console.log(ycommand);
                   
            };
        }();
        ax.baseaxes
            .on("mousedown", update_coords);

        ////////// DRAG POINT COMPONENT //////////
        
        var drag = d3.behavior.drag()
            .on("dragstart", dragstarted)
            .on("drag", dragged)
            .on("dragend", dragended);
            
        function dragstarted(d) {
             d3.event.sourceEvent.stopPropagation();
             d3.select(this).classed("dragging", true);
        }
        
        function dragged(d) {
             d3.select(this).attr("cx", d3.event.x)
                            .attr("cy", d3.event.y);             
        }

        function dragended(d, i) {
             d3.event.sourceEvent.stopPropagation();
             d3.select(this).classed("dragging", false);
             var calib_cx = d3.select(this)[0][0].cx.animVal.value - xcal;
             var calib_cy = d3.select(this)[0][0].cy.animVal.value - ycal;
             var xcommand = x+"["+i+"] = "+ax.x.invert(calib_cx);
             var ycommand = y+"["+i+"] = "+ax.y.invert(calib_cy);
             IPython.notebook.kernel.execute(xcommand);
             IPython.notebook.kernel.execute(ycommand);
             console.log(xcommand);
             console.log(ycommand);
        }


    };"""

    def __init__(self, radius=4, color="white", x ='x', y ='y'):
        self.dict_ = {"type": "annotate",
                      "radius": radius,
                      "color": color,
                      "x": x,
                      "y": y};
def pickpoints(fig='', radius=4, color="white", x = 'x', y = 'y'):
    mpld3.enable_notebook()
    if not fig:
        fig = plt.gcf()
    plugins.connect(fig, Annotate(radius, color, x, y)) # color='htmlcolorname', radius=int
    plugins.connect(fig, plugins.MousePosition())


In [ ]:
fig = plt.figure(figsize=(9,6))
plt.imshow(im)
pickpoints(fig, color='cyan', radius=2, x='x', y='y')

**STOP**: Use your mouse to select points in the above pictures. If the ```pickpoints``` function is working you should see small dots appear everywhere you click on the picture.  Pick points in a 3x3 grid pattern starting at the upper left corner and finishing in the lower right corner.

In [ ]:
#When you are done picking points turn off interaction to make the other figures work properly
mpld3.disable_notebook()

In [ ]:
#record source points
if x and y:
    src = np.array([x,y]).T

In [ ]:
# Construct Destination Image
width = 1000
dst = np.array([[0, width/2, width, 0, width/2, width, 0, width/2, width,], 
                [0, 0, 0, width/2, width/2, width/2, width, width, width]]).T

from skimage import transform
tform = transform.ProjectiveTransform()
tform.estimate(dst, src)
im2 = transform.warp(im, tform, output_shape=(1000,1000)) 


In [ ]:
#show original image next to transformed image
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(im)
ax1.scatter(src[:,0],src[:,1])
#Add numbers
for i in range(dst.shape[0]):
    ax1.annotate(str(i+1), (src[i,0]+20,src[i,1]), color='white');
ax1.set_title('Source Plain')

ax2.imshow(im2);
ax2.scatter(dst[:,0],dst[:,1])
#Add numbers
for i in range(dst.shape[0]):
    ax2.annotate(str(i+1), (dst[i,0]+20,dst[i,1]));
ax2.set_title('Destination Plain')
ax2.axis('off');


**&#9989;  DO THIS:** Think about ways you could automate the point selection process.  Discuss your ideas with your partner and be prepared to share with the group.

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with notes from your discussion.

**&#9989;  DO THIS:** Think about ways you could improve accuracy of this homography for this problem.  Discuss your ideas with your partner and be prepared to share with the group.

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with notes from your discussion.

----

# 2. Useful Tools for Image and Video Manipulation:

I would like to introduce all of you to two additional tools you may need for your research.  Both tools are Open Source and available for most computer platforms (Windows, Mac and Linux). They are typically run using your computers command line: 


**[ImageMagick](https://www.imagemagick.org/script/index.php):** This program is a library of image manipulation tools; the types of tools you would find in [Photoshop](https://www.adobe.com/products/photoshop.html) or [GIMP](https://www.gimp.org/).  Many of the more Artistic tools are not often used in scientific image analysis. However, some of them are quite useful. For example, the ```convert``` tool is very helpful in converting from one image type to another. For example, the following command converts ```myimage.jpg``` to ```myimage.png```:

```bash
convert myimage.jpg to myimage.png
```

A list of all image formats avaliable to ImageMagick (there are a lot of them) can be found here: https://www.imagemagick.org/script/formats.php


**[ffmpeg](https://www.ffmpeg.org/):** This program is amazing helpful when you are working with video files (ex. avi and mp4).  Similar to the ```convert``` command in ImageMagick the ```ffmpeg``` command can be used to convert video files from one format to another.  Also like ImageMagick, ffmpeg can handle a lot of different video formats (also called codex).  One of the most useful formats is a folder of images.  This means that ffmpeg can convert a video into a folder of images and can take a folder of images and turn it into a video.  For example, the following command converst an mp4 file into a folder of images:

```bash
ffmpeg -i MyMovie.mp4 -qscale 0 -f image2 -r 1 ./Images/img-%06d.png
```

Or you can go the other way. The following command converts a folder of images to a movie:

```bash
ffmpeg -r 60 -f image2 -i ./Images/img-%06d.png -vcodec libx264 -crf 25  -pix_fmt yuv420p MyMovie.mp4
```

----

# 3. Video Registration

**&#9989; DO THIS:** Consider the following two screen shots from two different cameras taken from the same experiment.  Think about the challenges for doing data registrations with these two images. Discuss these challenges with your partners and be prepared to share your discussion with the class.

In [ ]:
# Read data for this assignment
%matplotlib inline 
import matplotlib.pyplot as plt
from scipy.misc import imread, imsave
from urllib.request import urlopen

url1 = 'https://goo.gl/o7s8cz'
with urlopen(url1) as file:
    im1 = imread(file, mode='RGB')
url2 = 'https://goo.gl/pGhndC'
with urlopen(url2) as file:
    im2 = imread(file, mode='RGB')
    
f, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,10))
ax1.imshow(im1)
ax2.imshow(im2)

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with notes from your discussion.

----

# 4. Mid-Term Evaluation

Direct Link: https://goo.gl/tEHYcB

In [ ]:
from IPython.display import HTML
HTML(
"""
<iframe 
	src="https://goo.gl/tEHYcB?embedded=true" 
	width="100%" 
	height="1200px" 
	frameborder="0" 
	marginheight="0" 
	marginwidth="0">
	Loading...
</iframe>
"""
)

---------
### Congratulations, you're done with your in-class assignment!

&#169; Copyright 2018,  Michigan State University Board of Trustees